In [2]:
##patchcore学習完成###

import torch
from anomalib.data import Folder
from anomalib.models import Patchcore
#from anomalib.trainers import Trainer
from anomalib import TaskType
from anomalib.engine import Engine

# MPS（Apple GPU）を使用する設定
device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"

# データセットのパス設定
DATASET_PATH = "/Users/ken/Documents/ano_dev/anomalib/datasets/my_sample3/"
OUTPUT_PATH = "/Users/ken/Documents/ano_dev/anomalib/datasets/my_sample3/"

# データモジュールの設定
datamodule = Folder(
    name="my_sample",
    root="datasets/my_sample3",
    normal_dir="train/good",
    abnormal_dir="test/ng",
    #mask_dir="ground_truth",
    normal_test_dir="test/good",
    train_batch_size=32,
    eval_batch_size=32,
    task=TaskType.CLASSIFICATION,
    #task=TaskType.SEGMENTATION,
    #root=DATASET_PATH,
    #normal_dir="train/OK",
    #abnormal_dir="train/NG",
    image_size=(128, 128),  # 必要に応じて変更
)
datamodule.setup()

# PatchCoreモデルの設定
model = Patchcore(
    backbone="wide_resnet50_2",
    layers=("layer2", "layer3"),
    pre_trained=True,
    #backbone="mobilenetv3_large_100",  # 使用するバックボーンを指定
    #layers=("blocks.1","blocks.2","blocks.4","blocks.6"), # MobileNetV3の適切なレイヤーを指定
    #pre_trained=True,
    coreset_sampling_ratio=0.1,
    num_neighbors = 9
    #input_size=(256, 256),  # 必要に応じて変更
    #backbone="resnet18",    # 適切なバックボーンを選択
    #pre_trained=True,       # 事前学習済みモデルを使用
).to(device)  # モデルをMPSに転送

# トレーナーの設定
engine = Engine(
    max_epochs=50,  # エポック数を設定
    accelerator="auto",  # 自動的に適切なハードウェアを選択（MPSが優先される）
    devices=1,  # デバイス数を指定
    default_root_dir=OUTPUT_PATH,  # 出力フォルダ
)

# 学習実行
engine.fit(model, datamodule)
engine.test(datamodule=datamodule, model=model, verbose=True)


Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
/Users/ken/Documents/ano_dev/anomalib/.venv/lib/python3.11/site-packages/lightning/pytorch/core/optimizer.py:182: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer


┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name                  ┃ Type                     ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ model                 │ PatchcoreModel           │ 24.9 M │ train │
│ 1 │ _transform            │ Compose                  │      0 │ train │
│ 2 │ normalization_metrics │ MetricCollection         │      0 │ train │
│ 3 │ image_threshold       │ F1AdaptiveThreshold      │      0 │ train │
│ 4 │ pixel_threshold       │ F1AdaptiveThreshold      │      0 │ train │
│ 5 │ image_metrics         │ AnomalibMetricCollection │      0 │ train │
│ 6 │ pixel_metrics         │ AnomalibMetricCollection │      0 │ train │
└───┴───────────────────────┴──────────────────────────┴────────┴───────┘

Trainable params: 24.9 M                                                                                           
Non-trainable params: 0                                                                                            
Total params: 24.9 M                                                                                               
Total estimated model params size (MB): 99

/Users/ken/Documents/ano_dev/anomalib/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:419: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.
/Users/ken/Documents/ano_dev/anomalib/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:419: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Output()

...

`Trainer.fit` stopped: `max_epochs=1` reached.


F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
/Users/ken/Documents/ano_dev/anomalib/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:419: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │            1.0            │
│       image_F1Score       │            0.0            │
└───────────────────────────┴───────────────────────────┘

[{'image_AUROC': 1.0, 'image_F1Score': 0.0}]

In [3]:
##patchcore学習完成###

import torch
from anomalib.data import Folder
from anomalib.models import Patchcore
from anomalib import TaskType
from anomalib.engine import Engine

# MPS（Apple GPU）を使用する設定
device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"

# データセットのパス設定
DATASET_PATH = "/Users/ken/Documents/ano_dev/anomalib/datasets/my_sample3/"
OUTPUT_PATH = "/Users/ken/Documents/ano_dev/anomalib/datasets/my_sample3/"

# データモジュールの設定
datamodule = Folder(
    name="my_sample",
    root="datasets/my_sample3",
    normal_dir="train/good",
    abnormal_dir="test/ng",
    #mask_dir="ground_truth",
    normal_test_dir="test/good",
    train_batch_size=32,
    eval_batch_size=32,
    task=TaskType.CLASSIFICATION,
    #task=TaskType.SEGMENTATION,
    #root=DATASET_PATH,
    #normal_dir="train/OK",
    #abnormal_dir="train/NG",
    image_size=(128, 128),  # 必要に応じて変更
)
datamodule.setup()

# PatchCoreモデルの設定
model = Patchcore(
    backbone="wide_resnet50_2",
    layers=("layer2", "layer3"),
    pre_trained=True,
    coreset_sampling_ratio=0.1,
    num_neighbors = 9
).to(device)  # モデルをMPSに転送

# トレーナーの設定
engine = Engine(
    #max_epochs=50,  # エポック数を設定
    #accelerator="auto",  # 自動的に適切なハードウェアを選択（MPSが優先される）
    #devices=1,  # デバイス数を指定
    default_root_dir=OUTPUT_PATH,  # 出力フォルダ
)



In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from jsonargparse import Namespace
from anomalib.models import Patchcore
from anomalib.data import Folder
from anomalib import TaskType


predictions = engine.predict(
    model=model,
    ckpt_path = "/Users/ken/Documents/ano_dev/anomalib/datasets/my_sample3/Patchcore/my_sample/v5/weights/lightning/model.ckpt",
    data_path = "/Users/ken/Documents/ano_dev/anomalib/captured_images/",
    return_predictions=True
)

df = pd.DataFrame(predictions)
#print(df.columns)

#異常度
df["pred_scores"] = df["pred_scores"].apply(lambda x:x[0].item())
print(df["pred_scores"])
#############################
#規格前のknnの距離
df["org_anomaly_score"] = df["org_anomaly_score"].apply(lambda x:x.item())
print(df["org_anomaly_score"])
#############################
#異常度マップをnumpy arrayに変換
df["anomaly_maps"] = df["anomaly_maps"].apply(lambda x: np.array([[[b.item() for b in a] for a in z] for z in y] for y in x).squeeze())

save_df = df.loc[:,("image_path","pred_scores","org_anomaly_score")]

df["pred_scores"].plot.hist()
plt.savefig('/Users/ken/Documents/ano_dev/anomalib/.results/' + "/fig.png")
save_df.to_csv('/Users/ken/Documents/ano_dev/anomalib/.results/' + "/results.csv")


output_dir = "./results"  # 出力フォルダのパス
threshold = 0.5  # 閾値

# 結果CSVを読み込む
#results_csv = f"{output_dir}/results.csv"
results_csv = '/Users/ken/Documents/ano_dev/anomalib/.results/results.csv'
df = pd.read_csv(results_csv)

# 列名を確認
print(f"Columns in df: {df.columns}")

# 必要な列だけ取り出し、リスト化
results = []
for _, row in df.iterrows():
    try:
        image_path = row["image_path"]
        pred_score = row["pred_scores"]
        org_anomaly_score = row["org_anomaly_score"]
        result = "NG" if pred_score > threshold else "OK"

        # 結果を辞書形式で格納
        results.append({
            "Image Path": image_path,
            "Pred Score": pred_score,
            "Org Anomaly Score": org_anomaly_score,
            "Result": result
        })
    except KeyError as e:
        print(f"Missing column in row: {e}")

# 結果を表示
for idx, res in enumerate(results, start=0):
    print(f"Image {idx}: {res['Image Path']}")
    print(f"  Pred Score: {res['Pred Score']}")
    print(f"  Org Anomaly Score: {res['Org Anomaly Score']}")
    print(f"  Result: {res['Result']}")
    print("-" * 20)

# 結果をCSVファイルに保存
results_df = pd.DataFrame(results)
results_df.to_csv(f"{output_dir}/processed_results.csv", index=False)
print(f"Processed results saved to {output_dir}/processed_results.csv")

# 変数データのクリア
del df, results, results_df


Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
Restoring states from the checkpoint path at /Users/ken/Documents/ano_dev/anomalib/datasets/my_sample3/Patchcore/my_sample/v5/weights/lightning/model.ckpt
Loaded model weights from the checkpoint at /Users/ken/Documents/ano_dev/anomalib/datasets/my_sample3/Patchcore/my_sample/v5/weights/lightning/model.ckpt


Output()

/Users/ken/Documents/ano_dev/anomalib/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


0    0.514562
Name: pred_scores, dtype: float64
0    68.503387
Name: org_anomaly_score, dtype: float64
Columns in df: Index(['Unnamed: 0', 'image_path', 'pred_scores', 'org_anomaly_score'], dtype='object')
Image 0: ['/Users/ken/Documents/ano_dev/anomalib/captured_images/captured_image.jpg']
  Pred Score: 0.5145623087882996
  Org Anomaly Score: 68.50338745117188
  Result: NG
--------------------
Processed results saved to ./results/processed_results.csv


In [ ]:
#推論コマンドをAPIで実行。約2秒で処理できるのでこちらをベースに開発する
#初版ver1.0

from jsonargparse import Namespace
from tools.inference.my_lightning_inference import infer
import pandas as pd

output_dir = "./results"  # 出力フォルダのパス
threshold = 0.5  # 閾値

# 引数を設定
args = Namespace(
    ckpt_path="/Users/ken/Documents/ano_dev/anomalib/datasets/my_sample/Patchcore/my_sample/v25/weights/lightning/model.ckpt",
    data={"path": "/Users/ken/Documents/ano_dev/anomalib/datasets/my_sample/target1/"},
    output="/Users/ken/Documents/ano_dev/anomalib/results",
    #model="Patchcore",
    #    backbone="mobilenetv3_large_100",  # 使用するバックボーンを指定
    #layers=("blocks.1","blocks.2","blocks.4","blocks.6"), # MobileNetV3の適切なレイヤーを指定
    model = Patchcore(backbone="mobilenetv3_large_100", layers=["blocks.1", "blocks.2", "blocks.4", "blocks.6"]),
    threshold = 0.5,  # 閾値
    callbacks=None,
    show=False,
)

# 推論を実行
infer(args)


# 結果CSVを読み込む
results_csv = f"{output_dir}/results.csv"
df = pd.read_csv(results_csv)

# 必要な列だけ取り出し、リスト化
results = []
for _, row in df.iterrows():
    image_path = row["image_path"]
    pred_score = row["pred_scores"]
    org_anomaly_score = row["org_anomaly_score"]
    result = "NG" if pred_score > threshold else "OK"

    # 結果を辞書形式で格納
    results.append({
        "Image Path": image_path,
        "Pred Score": pred_score,
        "Org Anomaly Score": org_anomaly_score,
        "Result": result
    })

# 結果を表示
for idx, res in enumerate(results, start=1):
    print(f"Image {idx}: {res['Image Path']}")
    print(f"  Pred Score: {res['Pred Score']}")
    print(f"  Org Anomaly Score: {res['Org Anomaly Score']}")
    print(f"  Result: {res['Result']}")
    print("-" * 20)

# 結果をCSVファイルに保存
results_df = pd.DataFrame(results)
results_df.to_csv(f"{output_dir}/processed_results.csv", index=False)
print(f"Processed results saved to {output_dir}/processed_results.csv")

In [ ]:
#推論コマンドをAPIで実行。約2秒で処理できるのでこちらをベースに開発する
#OKNGのみ結果をリターン検討
#行けそう。開発ベースはこれ ver1.1

import time  # 時間計測のためのモジュール
from jsonargparse import Namespace
from tools.inference.my_lightning_inference import infer
import pandas as pd

def run_inference():
    output_dir = "./results"  # 出力フォルダのパス
    threshold = 0.9  # 閾値

    # 処理の開始時間を記録
    start_time = time.time()

    # 引数を設定
    args = Namespace(
        ckpt_path="/Users/ken/Documents/ano_dev/anomalib/datasets/my_sample/Patchcore/my_sample/v16/weights/lightning/model.ckpt",
        data={"path": "/Users/ken/Documents/ano_dev/anomalib/datasets/my_sample/target1/"},
        output="/Users/ken/Documents/ano_dev/anomalib/results",
        model="Patchcore",
        threshold=0.9,  # 閾値
        callbacks=None,
        show=False,
    )

    # 推論を実行
    infer(args)

    # 結果CSVを読み込む
    results_csv = f"{output_dir}/results.csv"
    df = pd.read_csv(results_csv)

    # 結果を取得（最初の画像のみ）
    row = df.iloc[0]  # 最初の画像の結果を使用
    pred_score = row["pred_scores"]

    # OK/NG を決定
    result = "NG" if pred_score > threshold else "OK"

    # 処理の終了時間を記録
    end_time = time.time()

    # 処理時間を計算
    elapsed_time = end_time - start_time
    print(f"Processing time: {elapsed_time:.2f} seconds")

    return result


In [ ]:
import pandas as pd
import subprocess

def main():
    # スクリプト実行設定
    inference_script = "tools/inference/my_lightning_inference.py"  # lightning_inference.pyのパス
    checkpoint_path = "/Users/ken/Documents/ano_dev/anomalib/datasets/my_sample/Patchcore/my_sample/v16/weights/lightning/model.ckpt"  # チェックポイントファイルのパス
    data_path = "/Users/ken/Documents/ano_dev/anomalib/datasets/my_sample/target/"  # データフォルダのパス
    output_dir = "./results"  # 出力フォルダのパス
    threshold = 0.5  # 閾値

    # lightning_inference.pyを実行
    cmd = [
        "python",
        inference_script,
        "--ckpt_path", checkpoint_path,
        "--data.path", data_path,
        "--output", output_dir,
        "--model", "Patchcore",
    ]
    subprocess.run(cmd, check=True)

    # 結果CSVを読み込む
    results_csv = f"{output_dir}/results.csv"
    df = pd.read_csv(results_csv)

    # 分岐処理
    for _, row in df.iterrows():
        image_path = row["image_path"]
        pred_score = row["pred_scores"]
        org_anomaly_score = row["org_anomaly_score"]

        print(f"Image: {image_path}")
        print(f"Pred Score: {pred_score}, Org Anomaly Score: {org_anomaly_score}")

        if pred_score > threshold:
            print(f"Result: NG (Anomaly detected for {image_path})")
        else:
            print(f"Result: OK (No significant anomaly for {image_path})")

if __name__ == "__main__":
    main()

In [ ]:
"""推論 CSVから読み込み"""
#コマンド起動のため遅いから没

import pandas as pd
import subprocess

def main():
    # スクリプト実行設定
    inference_script = "tools/inference/my_lightning_inference.py"  # lightning_inference.pyのパス
    checkpoint_path = "/Users/ken/Documents/ano_dev/anomalib/datasets/my_sample/Patchcore/my_sample/v16/weights/lightning/model.ckpt"  # チェックポイントファイルのパス
    data_path = "/Users/ken/Documents/ano_dev/anomalib/datasets/my_sample/target1/"  # データフォルダのパス
    output_dir = "./results"  # 出力フォルダのパス
    threshold = 0.9  # 閾値

    # lightning_inference.pyを実行
    cmd = [
        "python",
        inference_script,
        "--ckpt_path", checkpoint_path,
        "--data.path", data_path,
        "--output", output_dir,
        "--model", "Patchcore",
    ]
    subprocess.run(cmd, check=True)

    # 結果CSVを読み込む
    results_csv = f"{output_dir}/results.csv"
    df = pd.read_csv(results_csv)

    # 必要な列だけ取り出し、リスト化
    results = []
    for _, row in df.iterrows():
        image_path = row["image_path"]
        pred_score = row["pred_scores"]
        org_anomaly_score = row["org_anomaly_score"]
        result = "NG" if pred_score > threshold else "OK"

        # 結果を辞書形式で格納
        results.append({
            "Image Path": image_path,
            "Pred Score": pred_score,
            "Org Anomaly Score": org_anomaly_score,
            "Result": result
        })

    # 結果を表示
    for idx, res in enumerate(results, start=1):
        print(f"Image {idx}: {res['Image Path']}")
        print(f"  Pred Score: {res['Pred Score']}")
        print(f"  Org Anomaly Score: {res['Org Anomaly Score']}")
        print(f"  Result: {res['Result']}")
        print("-" * 20)

    # 結果をCSVファイルに保存
    results_df = pd.DataFrame(results)
    results_df.to_csv(f"{output_dir}/processed_results.csv", index=False)
    print(f"Processed results saved to {output_dir}/processed_results.csv")

if __name__ == "__main__":
    main()

In [ ]:
"""推論 メモリーから読み込み"""
#コマンド起動のため遅いから没

import pandas as pd
import subprocess
import io
import re

def main():
    # スクリプト実行設定
    inference_script = "tools/inference/my2_lightning_inference.py"  # lightning_inference.pyのパス
    checkpoint_path = "/Users/ken/Documents/ano_dev/anomalib/datasets/my_sample/Patchcore/my_sample/v16/weights/lightning/model.ckpt"  # チェックポイントファイルのパス
    data_path = "/Users/ken/Documents/ano_dev/anomalib/datasets/my_sample/target1/"  # データフォルダのパス
    output_dir = "./results"  # 出力フォルダのパス
    threshold = 0.9  # 閾値

    # lightning_inference.pyを実行
    cmd = [
        "python",
        inference_script,
        "--ckpt_path", checkpoint_path,
        "--data.path", data_path,
        "--output", output_dir,
        "--model", "Patchcore",
    ]
    
    try:
        # 標準出力とエラーをキャプチャ
        result = subprocess.run(cmd, check=True, text=True, capture_output=True)
    except subprocess.CalledProcessError as e:
        print(f"Subprocess failed with error: {e.stderr}")
        return

    # 標準出力から結果を抽出
    raw_output = result.stdout
    print("Raw output:\n", raw_output)  # デバッグ用に出力を表示

    # エスケープシーケンスを保持しつつ、CSV部分を抽出
    csv_pattern = r"(image_path,pred_scores,org_anomaly_score\n.+)"
    match = re.search(csv_pattern, raw_output, re.DOTALL)
    if not match:
        print("CSV data not found in the output.")
        return

    csv_data = match.group(1)

    # DataFrameに変換
    try:
        df = pd.read_csv(io.StringIO(csv_data))
    except pd.errors.ParserError as e:
        print(f"CSV parsing error: {e}")
        print("CSV data was:\n", csv_data)
        return

    print("Parsed DataFrame columns:", df.columns)

    # 必要な列だけ取り出し、リスト化
    results = []
    for _, row in df.iterrows():
        image_path = row["image_path"]
        pred_score = float(row["pred_scores"])
        org_anomaly_score = float(row["org_anomaly_score"])
        result = "NG" if pred_score > threshold else "OK"

        # 結果を辞書形式で格納
        results.append({
            "Image Path": image_path,
            "Pred Score": pred_score,
            "Org Anomaly Score": org_anomaly_score,
            "Result": result
        })

    # 結果を表示
    for idx, res in enumerate(results, start=1):
        print(f"Image {idx}: {res['Image Path']}")
        print(f"  Pred Score: {res['Pred Score']}")
        print(f"  Org Anomaly Score: {res['Org Anomaly Score']}")
        print(f"  Result: {res['Result']}")
        print("-" * 20)

    # 結果をCSVファイルに保存
    results_df = pd.DataFrame(results)
    results_df.to_csv(f"{output_dir}/processed_results.csv", index=False)
    print(f"Processed results saved to {output_dir}/processed_results.csv")

if __name__ == "__main__":
    main()